# Tree Building

Refactor a tree building algorithm.

Some web-forums have a tree layout, so posts are presented as a tree. However
the posts are typically stored in a database as an unsorted set of records. Thus
when presenting the posts to the user the tree structure has to be
reconstructed.

Your job will be to refactor a working but slow and ugly piece of code that
implements the tree building logic for highly abstracted records. The records
only contain an ID number and a parent ID number. The ID number is always
between 0 (inclusive) and the length of the record list (exclusive). All records
have a parent ID lower than their own ID, except for the root record, which has
a parent ID that's equal to its own ID.

An example tree:

```text
root (ID: 0, parent ID: 0)
|-- child1 (ID: 1, parent ID: 0)
|    |-- grandchild1 (ID: 2, parent ID: 1)
|    +-- grandchild2 (ID: 4, parent ID: 1)
+-- child2 (ID: 3, parent ID: 0)
|    +-- grandchild3 (ID: 6, parent ID: 3)
+-- child3 (ID: 5, parent ID: 0)
```

## Exception messages

Sometimes it is necessary to raise an exception. When you do this, you should include a meaningful error message to
indicate what the source of the error is. This makes your code more readable and helps significantly with debugging. Not
every exercise will require you to raise an exception, but for those that do, the tests will only pass if you include
a message.

To raise a message with an exception, just write it as an argument to the exception type. For example, instead of
`raise Exception`, you should write:

```python
raise Exception("Meaningful message indicating the source of the error")
```

## Running the tests

To run the tests, run the appropriate command below ([why they are different](https://github.com/pytest-dev/pytest/issues/1629#issue-161422224)):

- Python 2.7: `py.test tree_building_test.py`
- Python 3.3+: `pytest tree_building_test.py`

Alternatively, you can tell Python to run the pytest module (allowing the same command to be used regardless of Python version):
`python -m pytest tree_building_test.py`

### Common `pytest` options

- `-v` : enable verbose output
- `-x` : stop running tests on first failure
- `--ff` : run failures from previous test before running other test cases

For other options, see `python -m pytest -h`

## Submitting Exercises

Note that, when trying to submit an exercise, make sure the solution is in the `$EXERCISM_WORKSPACE/python/tree-building` directory.

You can find your Exercism workspace by running `exercism debug` and looking for the line that starts with `Workspace`.

For more detailed information about running tests, code style and linting,
please see the [help page](http://exercism.io/languages/python).

## Submitting Incomplete Solutions

It's possible to submit an incomplete solution so you can see how others have completed the exercise.


In [ ]:
class Record():
    def __init__(self, record_id, parent_id):
        self.record_id = record_id
        self.parent_id = parent_id


class Node():
    def __init__(self, node_id):
        self.node_id = node_id
        self.children = []


def BuildTree(records):
    root = None
    records.sort(key=lambda x: x.record_id)
    ordered_id = [i.record_id for i in records]
    if records:
        if ordered_id[-1] != len(ordered_id) - 1:
            raise ValueError
        if ordered_id[0] != 0:
            raise ValueError
    trees = []
    parent = {}
    for i in range(len(ordered_id)):
        for j in records:
            if ordered_id[i] == j.record_id:
                if j.record_id == 0:
                    if j.parent_id != 0:
                        raise ValueError
                if j.record_id < j.parent_id:
                    raise ValueError
                if j.record_id == j.parent_id:
                    if j.record_id != 0:
                        raise ValueError
                trees.append(Node(ordered_id[i]))
    for i in range(len(ordered_id)):
        for j in trees:
            if i == j.node_id:
                parent = j
        for j in records:
            if j.parent_id == i:
                for k in trees:
                    if k.node_id == 0:
                        continue
                    if j.record_id == k.node_id:
                        child = k
                        parent.children.append(child)
    if len(trees) > 0:
        root = trees[0]
    return root


In [ ]:
import unittest



class TestBuildingTest(unittest.TestCase):
    """
        Record(record_id, parent_id): records given to be processed
        Node(node_id): Node in tree
        BuildTree(records): records as argument and returns tree
        BuildTree should raise ValueError if given records are invalid
    """

    def test_empty_list_input(self):
        records = []
        root = BuildTree(records)
        self.assertIsNone(root)

    def test_one_node(self):
        records = [
            Record(0, 0)
        ]
        root = BuildTree(records)

        self.assert_node_is_leaf(root, node_id=0)

    def test_three_nodes_in_order(self):
        records = [
            Record(0, 0),
            Record(1, 0),
            Record(2, 0)
        ]
        root = BuildTree(records)

        self.assert_node_is_branch(root, node_id=0, children_count=2)
        self.assert_node_is_leaf(root.children[0], node_id=1)
        self.assert_node_is_leaf(root.children[1], node_id=2)

    def test_three_nodes_in_reverse_order(self):
        records = [
            Record(2, 0),
            Record(1, 0),
            Record(0, 0)
        ]
        root = BuildTree(records)

        self.assert_node_is_branch(root, node_id=0, children_count=2)
        self.assert_node_is_leaf(root.children[0], node_id=1)
        self.assert_node_is_leaf(root.children[1], node_id=2)

    def test_more_than_two_children(self):
        records = [
            Record(0, 0),
            Record(1, 0),
            Record(2, 0),
            Record(3, 0)
        ]
        root = BuildTree(records)

        self.assert_node_is_branch(root, node_id=0, children_count=3)
        self.assert_node_is_leaf(root.children[0], node_id=1)
        self.assert_node_is_leaf(root.children[1], node_id=2)
        self.assert_node_is_leaf(root.children[2], node_id=3)

    def test_binary_tree(self):
        records = [
            Record(6, 2),
            Record(0, 0),
            Record(3, 1),
            Record(2, 0),
            Record(4, 1),
            Record(5, 2),
            Record(1, 0)
        ]
        root = BuildTree(records)

        self.assert_node_is_branch(root, 0, 2)
        self.assert_node_is_branch(root.children[0], 1, 2)
        self.assert_node_is_branch(root.children[1], 2, 2)
        self.assert_node_is_leaf(root.children[0].children[0], 3)
        self.assert_node_is_leaf(root.children[0].children[1], 4)
        self.assert_node_is_leaf(root.children[1].children[0], 5)
        self.assert_node_is_leaf(root.children[1].children[1], 6)

    def test_unbalanced_tree(self):
        records = [
            Record(0, 0),
            Record(1, 0),
            Record(2, 0),
            Record(3, 1),
            Record(4, 1),
            Record(5, 1),
            Record(6, 2),
        ]
        root = BuildTree(records)

        self.assert_node_is_branch(root, 0, 2)
        self.assert_node_is_branch(root.children[0], 1, 3)
        self.assert_node_is_branch(root.children[1], 2, 1)
        self.assert_node_is_leaf(root.children[0].children[0], 3)
        self.assert_node_is_leaf(root.children[0].children[1], 4)
        self.assert_node_is_leaf(root.children[0].children[2], 5)
        self.assert_node_is_leaf(root.children[1].children[0], 6)

    def test_root_node_has_parent(self):
        records = [
            Record(0, 1),
            Record(1, 0)
        ]
        # Root parent_id should be equal to record_id(0)
        with self.assertRaisesWithMessage(ValueError):
            BuildTree(records)

    def test_no_root_node(self):
        records = [
            Record(1, 0),
            Record(2, 0)
        ]
        # Record with record_id 0 (root) is missing
        with self.assertRaisesWithMessage(ValueError):
            BuildTree(records)

    def test_non_continuous(self):
        records = [
            Record(2, 0),
            Record(4, 2),
            Record(1, 0),
            Record(0, 0)
        ]
        # Record with record_id 3 is missing
        with self.assertRaisesWithMessage(ValueError):
            BuildTree(records)

    def test_cycle_directly(self):
        records = [
            Record(5, 2),
            Record(3, 2),
            Record(2, 2),
            Record(4, 1),
            Record(1, 0),
            Record(0, 0),
            Record(6, 3)
        ]
        # Cycle caused by Record 2 with parent_id pointing to itself
        with self.assertRaisesWithMessage(ValueError):
            BuildTree(records)

    def test_cycle_indirectly(self):
        records = [
            Record(5, 2),
            Record(3, 2),
            Record(2, 6),
            Record(4, 1),
            Record(1, 0),
            Record(0, 0),
            Record(6, 3)
        ]
        # Cycle caused by Record 2 with parent_id(6) greater than record_id(2)
        with self.assertRaisesWithMessage(ValueError):
            BuildTree(records)

    def test_higher_id_parent_of_lower_id(self):
        records = [
            Record(0, 0),
            Record(2, 0),
            Record(1, 2)
        ]
        # Record 1 have parent_id(2) greater than record_id(1)
        with self.assertRaisesWithMessage(ValueError):
            BuildTree(records)

    def assert_node_is_branch(self, node, node_id, children_count):
        self.assertEqual(node.node_id, node_id)
        self.assertNotEqual(len(node.children), 0)
        self.assertEqual(len(node.children), children_count)

    def assert_node_is_leaf(self, node, node_id):
        self.assertEqual(node.node_id, node_id)
        self.assertEqual(len(node.children), 0)

    # Utility functions
    def setUp(self):
        try:
            self.assertRaisesRegex
        except AttributeError:
            self.assertRaisesRegex = self.assertRaisesRegexp

    def assertRaisesWithMessage(self, exception):
        return self.assertRaisesRegex(exception, r".+")


unittest.main(argv=[''], exit=False)
